In [1]:
import os
import PyPDF2
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = " ".join(page.extract_text() or "" for page in reader.pages)
    return text

In [4]:
resume_dir = "resumes/"
# jd_path = "JDs/AI_intern_Armada.txt"
jd_path = "JDs/Data_engineering_intern_LiveRamp.txt"
with open(jd_path, 'r', encoding='utf-8') as f:
    job_description = f.read()

jd_embedding = model.encode(job_description, convert_to_tensor=True)



In [5]:
results = []

for file_name in os.listdir(resume_dir):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(resume_dir, file_name)
        resume_text = extract_text_from_pdf(file_path)
        resume_embedding = model.encode(resume_text, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(resume_embedding, jd_embedding).item()
        score = round(similarity * 100, 2)
        
        results.append({
            "Resume": file_name,
            "Similarity Score": score
        })

In [6]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Similarity Score", ascending=False).reset_index(drop=True)

In [7]:
results_df

,Resume,Similarity Score
0,Dhruvraj_resume_USHunger.pdf,53.87
1,Dhruvraj_Resume_intern_rocket.pdf,53.45
2,Dhruvraj_resume_May18.pdf,44.75
3,Dhruvraj_resume_Mar11.pdf,40.43
4,Dhruvraj_resume_MSDS.pdf,35.98
5,Dhruvraj_resume_image_analytics.pdf,25.18
